Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
%%time
import pandas as pd
ratings = pd.read_csv('ml-25m/ratings.csv')

Wall time: 11.7 s


In [2]:
%%time
groupped = ratings.groupby('movieId').mean()[['rating']]
groupped.head()

Wall time: 3.1 s


,rating
movieId,
1,3.893708
2,3.251527
3,3.142028
4,2.853547
5,3.058434


In [3]:
import numpy as np
def get_rating(movieId):
    try:
        rating = groupped.loc[movieId][0]
    except KeyError as ke:
#         print(f'KeyError: {ke}')
        return np.NaN

    if rating <= 2:
        return 'low'
    if rating <= 4:
        return 'average'
    if rating >= 4.5:
        return 'high'
    return None

In [4]:
movies = pd.read_csv('ml-25m/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
movies['class'] = movies.movieId.apply(get_rating)
movies.head()

,movieId,title,genres,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,average
1,2,Jumanji (1995),Adventure|Children|Fantasy,average
2,3,Grumpier Old Men (1995),Comedy|Romance,average
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,average
4,5,Father of the Bride Part II (1995),Comedy,average


Задание 2

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.

In [6]:
keywords = pd.read_csv('keywords.csv')

geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [7]:
def get_region(keyword):
    for region, cities in geo_data.items():
        for city in cities:
            if keyword.lower().find(city) > -1:
                return region
    return 'undefined'

keywords['region'] = keywords.keyword.apply(get_region)

keywords[(keywords.region == 'Центр') | (keywords.region == 'Дальний Восток')].iloc[:10]

,keyword,shows,region
127,авито москва,979292,Центр
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
1259,москва,153050,Центр
1396,гисметео москва,140279,Центр
1482,госуслуги москва,133061,Центр
1579,авито тула,126168,Центр
1682,погода москва,151701,Центр
2175,дром хабаровск,98361,Дальний Восток
2806,циан недвижимость москва,78729,Центр


Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

    В переменную years запишите список из всех годов с 1950 по 2010.

    Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
        для каждой строки пройдите по всем годам списка years
        если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
        если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

    Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

    Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [8]:
years = [str(year) for year in range(1950, 2011)]
    
def production_year(title):
    for year in years:
        if title.find(year) > -1:
            return int(year)
    return 1900

def get_rating_mean(movieId):
    try:
        rating = groupped.loc[movieId][0]
        return rating
    except:
        return np.NaN

movies['year'] = movies.title.apply(production_year)
movies['rating'] = movies.movieId.apply(get_rating_mean)
movies.head()

,movieId,title,genres,class,year,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,average,1995,3.893708
1,2,Jumanji (1995),Adventure|Children|Fantasy,average,1995,3.251527
2,3,Grumpier Old Men (1995),Comedy|Romance,average,1995,3.142028
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,average,1995,2.853547
4,5,Father of the Bride Part II (1995),Comedy,average,1995,3.058434


In [9]:
movies.groupby('year')[['rating']].mean().sort_values('rating', ascending=False)

,rating
year,
1963,3.228859
1961,3.208200
1962,3.194438
1953,3.190671
1950,3.179434
...,...
1991,2.973292
1990,2.955692
1987,2.941741


In [11]:
movies[movies.year == 1963].sort_values('rating', ascending=False).head(10)

,movieId,title,genres,class,year,rating
37397,152559,Tammy and the Doctor (1963),Comedy|Romance,high,1963,5.00
35567,148122,Son of Hercules vs. Medusa (1963),Adventure|Fantasy,high,1963,5.00
34896,146489,Hercules Against the Mongols (1963),Action|Adventure|Drama,high,1963,5.00
30051,135350,Toto vs the Four (1963),Comedy,high,1963,5.00
29033,133017,Samson and the Slave Queen (1963),Action|Adventure|Drama,high,1963,4.50
55783,192715,A Stitch in Time (1963),Comedy,high,1963,4.50
55271,191529,Symphonie pour un massacre (1963),Crime|Drama,high,1963,4.50
44474,168302,The Red Lanterns (1963),Drama,None,1963,4.25
58236,198183,The Caretaker (1963),Drama,None,1963,4.25
60770,204114,Le glaive et la balance (1963),(no genres listed),None,1963,4.25
